In [4]:
!pip install catboost -q

In [5]:
from catboost import CatBoostClassifier, Pool
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
import pandas as pd

In [6]:
data = pd.read_parquet("/kaggle/input/ecup-part-2/dataset.parquet", engine='pyarrow')

In [7]:
!wget http://79.137.199.173/datasets/ozon_tech/features_full.csv

--2024-08-26 18:59:18--  http://79.137.199.173/datasets/ozon_tech/features_full.csv
Connecting to 79.137.199.173:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178632335 (170M) [text/csv]
Saving to: 'features_full.csv'

features_full.csv   100%[===================>] 170.36M  24.8MB/s    in 7.8s    

2024-08-26 18:59:26 (22.0 MB/s) - 'features_full.csv' saved [178632335/178632335]



In [8]:
features = pd.read_csv("/kaggle/working/features_full.csv")

In [10]:
data = data.merge(features, left_on='variantid1', right_on='variantid', how='left')
data = data.drop(['variantid'], axis=1)
data = data.merge(features, left_on='variantid2', right_on='variantid', how='left')
data = data.drop(['variantid'], axis=1)

In [ ]:
model = CatBoostClassifier(eval_metric = "PRAUC", early_stopping_rounds=100,verbose=100, boosting_type = "Plain")

train_data = data.drop(['variantid1', 'variantid2', 'target'], axis = 1)
train_labels = data['target']
grid = {
    'learning_rate': [0.03, 0.07, 0.1],  
    "l2_leaf_reg":[3, 5, 7, 9],
    "colsample_bylevel": [0.65, 0.83, 0.95],
    "depth": [4, 6, 9],
    "min_data_in_leaf": [40, 100, 160],
    "max_bin": [160, 255, 340],
    "bootstrap_type":  ["Bernoulli", "MVS"],
    "subsample": [0.75, 0.82, 0.91]
   # "eval_metric": "Precision"
}



grid_search_result = model.randomized_search(grid,
                                       X=train_data,
                                       y=train_labels,
                                       n_iter=8,
                                       cv=3,                
                                       partition_random_seed=42,  
                                       refit=True,                  
                                       plot=True)            



best_model = model

In [ ]:
best_model.get_params()

In [ ]:
best_model.save_model("catboost_model_rd.cbm", format="cbm")